# Finde ein effizientes Programm um eine Floyd Rose Gitarre zu stimmen

## 1. Einleitung

Floyd Rose Gitarren sind eine spezielle Art von E-Gitarren, die eine spezielle Art von Brücke verwenden, die als Floyd Rose Tremolo bekannt ist. Diese Brücke ermöglicht es dem Gitarristen, die Tonhöhe der Saiten zu ändern, indem er einen Hebel betätigt, der an der Brücke befestigt ist. Dies ermöglicht es dem Gitarristen, spezielle Effekte wie Vibrato und Divebombs zu erzeugen. Es erschwert jedoch auch das Stimmen der Gitarre erheblich, da bei dem Stimmen einer Saite die anderen Saiten ebenfalls verstimmt werden können, da die Spannung der Saiten auf der Brücke verteilt ist. Ein Grund dafür ist das die Saiten an Federn aufgehängt sind. Dies kann dazu führen, dass das Stimmen einer Floyd Rose Gitarre sehr zeitaufwändig ist.

In dieser Arbeit stellen wir zunächst ein Experiment auf, welches uns informationen über das Verhalten der Saiten gibt, wenn wir eine Saite stimmen. Mit diesen Informationen leiten wir ein mathematisches Modell ab, welches uns erlaubt, die Saiten einer Floyd Rose Gitarre effizient zu stimmen. Es soll berechnet werden welche Saiten in welcher Reihenfolge zu welcher Frequenz gestimmt werden müssen, um die Gitarre möglichst schnell und genau zu stimmen.

Auf dem Weg zu diesem Ziel werden wir ein Programm erstellen, welches uns erlaubt die Frequenz von Saiten zu messen. Die Messdaten von dem Experiment werden Visualisiert und anhand dessen wird das mathematische Modell erstellt.

Anschließend wird ein Program erstellt welches berechnet, welche Saiten in welcher Reihenfolge zu welcher Frequenz gestimmt werden müssen, um die Gitarre möglichst schnell und genau zu stimmen.

Die Arbeit wird in folgende Schritte unterteilt:
1. Experiment
2. Visualisierung
3. Mathematisches Modell
4. Programmierung
5. Testen
6. Optimierung
7. Weiterführendes

## 2. Experiment

Um die Saiten einer Floyd Rose Gitarre effizient zu stimmen, müssen wir zunächst ein Experiment durchführen, welches uns Informationen über das Verhalten der Saiten gibt, wenn wir eine Saite stimmen.

Dazu wird zunächst jede Saite in eine Ausgangsposition gebracht. Eine Standard Gitarren Stimmung ist EADGBE. Die Saiten werden so gestimmt, dass sie die Frequenzen E2, A2, D3, G3, B3 und E4 haben. Es handelt sich hierbei um die gleich temperierte Stimmung[1].
Die Saiten werden demnach die Frequenzen haben:\
E2 = 82.41 Hz\
A2 = 110.00 Hz\
D3 = 146.83 Hz\
G3 = 196.00 Hz\
B3 = 246.94 Hz\
E4 = 329.63 Hz

Dann wird jede Saite um ein Bestimmtes delta verstimmt. Das Delta wird so gewählt, dass die Frequenz der Saite um 20 Cent[2] erhöht wird. Die Frequenz der Saiten wird gemessen und aufgezeichnet.

delta = 20 Cent

Die Einheit Cent wird gewählt, da sie eine logarithmische Einheit ist, die in der Musik verwendet wird, um die Tonhöhe von Tönen zu beschreiben. Ein Cent entspricht einem Hundertstel eines Halbtons.

Jede Saite wird um 10 delta Schritte jeweils nach oben und nach unten verstimmt. Für jeden Schritt wird die Frequenz der anderen Saiten gemessen und aufgezeichnet.

Schritte in beide Richtungen = 10

Das Ziel des Experiments ist zu beobachten ob das System
- linear ist
- elastisch ist

Die erkenntnisse bestimmen später das mathematische Modell.

### 2.1 Materialien
- Floyd Rose Gitarre
- Frequenz Messgerät
- Visualisierungssoftware

Die Floyd Rose Gitarre ist verfügbar. Das Frequenz Messgerät wird ein Python Programm sein, welches die Frequenz der Saiten misst. Die Visualisierungssoftware wird ebenfalls ein Python Programm sein, welches die Messdaten visualisiert.

#### 2.1.1 Frequenz Messgerät

Gegeben sei eine Liste von Samples. Diese Liste wird mit Hilfe der Fourier Transformation in den Frequenzbereich transformiert. Die Frequenz mit der höchsten Amplitude wird als die Frequenz der Saite angenommen.


In [71]:
from scipy.fftpack import fft
from scipy.io import wavfile  # Import the wavfile API
import numpy as np

def get_samples(filepath):
    fs, data = wavfile.read(filepath)  # Load the data
    if len(data.shape) > 1:  # Check if stereo
        data = data.mean(axis=1)  # Convert to mono by averaging channels
    return data, fs

def fourier_transform(samples):
    # Normalize the data
    max_val = 2 ** (8 * samples.itemsize - 1)
    normalized_data = samples / max_val
    # Calculate the Fourier transform
    spectrum = fft(normalized_data)
    half = len(spectrum) // 2  # Take the first half of the spectrum
    return spectrum[:half]

def get_peak_frequency(spectrum, samplerate):
    # Get the magnitude of the spectrum
    magnitude = np.abs(spectrum)
    # Find the index of the peak frequency
    peak_idx = np.argmax(magnitude)
    # Map the index to a frequency value
    frequency = peak_idx * (samplerate / (2 * len(spectrum)))
    return frequency

def get_tone_from_file(filepath):
    samples, samplerate = get_samples(filepath)
    spectrum = fourier_transform(samples)
    return get_peak_frequency(spectrum, samplerate)

#### 2.1.2 Cent Messgerät
Da wir nun die Frequenz Messen können, müssen wir den Unterschied zweier Frequenzen in Cent umwandeln. Dazu wird die folgende Funktion erstellt.

In [72]:
def frequency_difference_to_cent(f1,f2):
    return 1200 * np.log2(f1/f2) # 1200 Cent entsprechen einem Halbton


#### 2.1.2 Visualisierungssoftware

Mit hilfe von matplotlib wird eine Visualisierung erstellt. Die Visualisierung zeigt die Frequenz der Saiten in Abhängigkeit von der Verstimmung.
Die daten zum Visualisieren werden wie folgt strukturiert:

```
{
    'E2': {
        'E2': array([-10,...,10]),
        'D3': array([record_-10,record_-9,...,record_10]),
        ...,
        'E4': array([records_per_step,...])
    },
    ...
    'E4': {
        'E2': array([records_per_step,...]),
        'A2': array([records_per_step,...]),
        ...,
        'E4': array([-10,...,10])
    }
}
```

Das folgende Programm ermöglicht es uns, die Daten zu visualisieren. Es wird eine Checkbox für jede Saite erstellt, die es uns ermöglicht, die Daten für jede Saite einzeln anzuzeigen. Die Visualisierung zeigt die Auswirkungen der Verstimmung einer Saite auf die Frequenz der anderen Saiten.

In [73]:
import matplotlib.pyplot as plt
from ipywidgets import  HBox, VBox, interactive, Layout, Checkbox, fixed
import random

nStrings = 6
steps = 10
step_range = range(-steps, steps+1) # -10 bis 10
strings = ["E2", "A2", "D3", "G3", "B3", "E4"]
# Verstimmung jeder anderen seite für jede Saite für alle Verstimmungsschritte
example_dataset = { # Dummy Data
    string: {other_string: np.array([ -_*random.random()if other_string != string else _ for _ in step_range]) for other_string in strings}
    for string in strings
}


def visualisation(df,string,**args):
    for other_string in strings:
        if other_string in args.keys() and args[other_string]:
            plt.plot(step_range, df[string][other_string], label=other_string)
    plt.title("Impact on String when detuning String "+string)
    plt.xlabel(f"Detuning of String {string} in Cent")
    plt.ylabel(f"Frequency of other Strings in Cent")
    plt.legend()


def visualize_all(data):
    widget_list = []

    for string in strings:
        checkboxes = {string: Checkbox(value=True,label=string,indent=False) for string in strings}
        widget = interactive(visualisation,df=fixed(data),string=fixed(string),**checkboxes)
        controls = HBox(widget.children[:-1]) # Horizontale Box für die Checkboxes
        output = widget.children[-1]
        w=VBox([controls, output], layout=Layout(margin="10px"))
        widget_list.append(w)
    row1 = HBox(widget_list[:3])
    row2 = HBox(widget_list[3:])
    output = VBox([row1, row2])
    display(output)



visualize_all(example_dataset)


### 2.2 Durchführung

In der Live Demo wird das Experiment durchgeführt. Die Frequenz der Saiten wird gemessen und die Daten werden visualisiert.
Hier werden nun vorab aufgenommene Daten verwendet. Die Audio Dateien sind im Ordner `audio` zu finden.
Die Ordnerstruktur ist wie folgt:

    ```
    audio/<variable_saite>/<beeinflusste_saite>/<verstimmungsschritt>.wav

    audio
    ├── E2 # Ändernde Saite
      ├── E2 # Beinflusste Saite (Trivialer Fall da die Frequenz der Saite sich nicht ändert)
        ├─── -10.wav
        ├─── ...
        ├─── 10.wav
      ├─── ...
      ├─── E4
    ├── ...
    ├── E4

In [ ]:
# Experiment
measured_data = {}
filenames =  [f"{i}.wav" for i in step_range]
original_frequencies = [82.41, 110.00, 146.83, 196.00, 246.94, 329.63]
for changing_string in strings:
    for impacted_string in strings:
        for filename in filenames:
            samples = get_samples(f"audio/{changing_string}/{impacted_string}/{filename}")
            frequency = get_tone_from_samples(samples)
            f0 = original_frequencies[strings.index(changing_string)]
            cent = frequency_difference_to_cent(f0, frequency)

            print(f"Saite: {changing_string}, Beeinflusste Saite: {impacted_string}, Verstimmungsschritt: {filename}, Verstimmung in Cent {cent}, Frequenz: {frequency}")

            if changing_string not in measured_data:
                measured_data[changing_string] = {}

            if impacted_string not in measured_data[changing_string]:
                measured_data[changing_string][impacted_string] = []

            measured_data[changing_string][impacted_string].append(cent)

## 3. Visualisierung

In [75]:
visualize_all(measured_data)

### 4.1 Mathematisches Modell

Die Messdaten zeigen dass das System linear ist. Das bedeutet, dass die Verstimmung einer Saite die Frequenz der anderen Saiten linear beeinflusst. Das System ist auch elastisch, was bedeutet, dass die Frequenz einer Saite wieder in ihren Ausgangszustand zurückkehrt, wenn die Verstimmung aufgehoben wird.

Wir können das System als ein lineares Gleichungssystem beschreiben. Die Frequenz der Saiten kann als Vektor dargestellt werden. Die Verstimmung der Saiten kann als Matrix dargestellt werden. Die Verstimmung der Saiten ist eine lineare Transformation der Frequenz der Saiten.

$$
\vec{f_0} = \begin{pmatrix} {f_{E2}} \\ {f_{A2}} \\ {f_{D3}} \\ {f_{G3}} \\ {f_{B3}} \\ {f_{e4}} \end{pmatrix},
C = \begin{bmatrix} {1} & { c_{12} } & {c_{13}} & {c_{14}} & {c_{15}} & {c_{16}} \\ {c_{21}} & {1} & {c_{23}} & {c_{24}} & {c_{25}} & {c_{26}} \\ {c_{31}} & {c_{32}} & {1} & {c_{34}} & {c_{35}} & {c_{36}} \\ {c_{41}} & {c_{42}} & {c_{43}} & {1} & {c_{45}} & {c_{46}} \\ {c_{51}} & {c_{52}} & {c_{53}} & {c_{54}} & {1} & {c_{56}} \\ {c_{61}} & {c_{62}} & {c_{63}} & {c_{64}} & {c_{65}} & {1} \end{bmatrix},
\vec{g} = \begin{pmatrix} {\hat{f}_{E2}} \\ {\hat{f}_{A2}} \\ {\hat{f}_{D3}} \\ {\hat{f}_{G3}} \\ {\hat{f}_{B3}} \\ {\hat{f}_{e4}} \end{pmatrix}
$$
\
$\vec{f_0}$ sind die Frequenzen der Saiten in der Ausgangsposition.\
$C$ ist die Verstimmungsmatrix. Die Elemente $c_{ij}$ sind die Verstimmungsfaktoren der Saite $i$, wenn die Saite $j$ um 1 Schritt (20 Cent) verstimmt wird.\
$\vec{g}$ sind die Frequenzen der Saiten nach der Verstimmung.\
$\vec{g}$ wollen wir im allgemeinen als Zielfrequenzen erreichen.

Die Annahme ist, es gibt eine Verstimmung $\vec{\delta}$ der Saiten, die die Frequenzen der Saiten in $\vec{g}$ erreicht, wenn sie dem System $C$ unterworfen wird.

$$
\vec{g} = add\_cent\_to\_freq(C \cdot \vec{\delta}, \vec{f_0})
$$

$$
\vec{\delta} = cent(\vec{f_0}-\vec{g}) \cdot C^{-1}
$$

$C^{-1}$ ist die Inverse der Verstimmungsmatrix.\
$cent$ ist die Funktion, die die Frequenzdifferenz in Cent umwandelt.\
$add\_cent\_to\_freq$ ist die Funktion, die die Verstimmung in Cent zu den Frequenzen der Saiten addiert.

### 5. Programmierung
Im folgendem werden die Funktionen `cent`, `add_cent_to_freq` und `find_coefficients` implementiert.

#### 5.1 Funktionen




In [ ]:
def cent(f1,f2):
    return 1200 * np.log2(f1/f2) # 1200 Cent entsprechen einer Oktave

def add_cent_to_freq(cent,freq):
    return freq * 2 ** (cent/1200)/12

def find_coefficients(f0, g, C):
    return (g-f0) @ np.linalg.inv(C)


## Literatur
[1]Gleich temperierte Stimmung: https://de.wikipedia.org/wiki/Gleichstufige_Stimmung \
[2]Cent: https://de.wikipedia.org/wiki/Cent_(Musik)

